## Including Fees in transaction configuration
In this example we demonstrate the configuration of transaction types that take fees into consideration. We have included an example of adding a non-capitalised fee (it is not considered in the cost basis of the resultant holding so we have to create a carry movement to account for it).

## Imports


In [7]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
from IPython.core.display import HTML
from datetime import datetime, timedelta
import logging
logging.basicConfig(level=logging.INFO)

import lusid as lu
import lusid.api as la
import lusid.models as lm

from lusidjam import RefreshingToken
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.jupyter_tools import StopExecution
from finbourne_sdk_utils.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,apiVersion,buildVersion,excelVersion,links
0,v0,0.6.14400.0,0.5.3666,"{'relation': 'RequestLogs', 'href': 'https://n..."


In [8]:
instruments_api = api_factory.build(la.InstrumentsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)
transaction_config_api = api_factory.build(la.TransactionConfigurationApi)

In [9]:
#setting up variables

scope = "FeesTesting"
code = "FeesTestV1"
print(f"'{scope}/{code}' scope and code created.")

custom_transaction_type = "TestType"
custom_side = "TestSide"

'FeesTesting/FeesTestV1' scope and code created.


## 1. Create instruments, portfolio and transactions to work with

### 1.0 Load Transaction Data

In [10]:
#reading transactions file with a funds in transaction and a buy of BP
#using example source file

transactions_df = pd.read_csv("data/commission-txns.csv", keep_default_na = False)
transactions_df.index += 1
display(transactions_df)

,instrument,asset,figi,txn_id,txn_type,trade_date,settle_date,units,price,currency,commission
1,GBP,Cash,,3007,FundsIn,2022-06-06T09:00:00Z,2022-06-08T09:00:00Z,500,1,GBP,0.00
2,BP,Equity,BBG000C05BD1,3008,Buy,2022-06-06T11:00:00Z,2022-06-08T09:00:00Z,10,5,GBP,1.50


### 1.1 Load Instruments

#### Loaded from df file

In [11]:
import os
print(os.getenv("FBN_ACCESS_TOKEN"))

MTdkZTAxODFjMjhhNDUwMGJmZWM1YjVhYTE0ZjBlMmN8NzA4ZTdjNDYxZmI5Yzg5YzU1Yjc5MjcyMzFlMGY2YmMyZWM0MTgwMjMwOTAyZmJlODZhYjM0YTBjMjQ1OTliMA==


In [6]:

definitions = {
    security["instrument"]: lm.InstrumentDefinition(
            name = security["instrument"],
            identifiers = {
                "Figi": lm.InstrumentIdValue(value = security["figi"]),
            },
            definition = lm.Equity(
                instrument_type = "Equity",
                dom_ccy = security["currency"],
                identifiers = {}
            )
        )
    for index, security in transactions_df.iterrows() if security["asset"] == "Equity"
}

upsert_instruments_response = instruments_api.upsert_instruments(
    request_body = definitions,
    scope = f"{scope}{code}",
)

upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instruments_response.values.values()))
display(upsert_instruments_response_df[["name", "lusidInstrumentId"]])

,name,lusidInstrumentId
0,BP,LUID_00003D68


### 1.2 Create Portfolio

In [12]:

portfolio_definition=lm.CreateTransactionPortfolioRequest(
    display_name="Fees Test Portfolio",
    code = code,
    base_currency = "GBP",
    created="2020-01-01T00:00:00Z",
    instrument_scopes = [f"{scope}{code}"],
)

try:
    create_portfolio_response=transaction_portfolios_api.create_portfolio(
        scope = scope,
        create_transaction_portfolio_request = portfolio_definition
    )
    print(f"Portfolio with display name '{create_portfolio_response.display_name}' created effective {str(create_portfolio_response.created)}")
except lu.ApiException as e:
    if json.loads(e.body)["name"] == "PortfolioWithIdAlreadyExists":
            logging.info(json.loads(e.body)["title"])

INFO:root:Could not create a portfolio with id 'FeesTestV1' because it already exists in scope 'FeesTesting'.


## 2. Create Transaction Properties for fees

#### In this case we use derived properties

In [13]:

property_definition = lm.CreateDerivedPropertyDefinitionRequest(
    domain="Transaction",  
    scope="Fees",  
    code="NonCapFee1",  
    data_type_id=lm.ResourceId(
        scope="system",
        code="number"  
    ),
    derivation_formula="round(Units * TransactionPrice.Price * 0.1, 0.1)",
    display_name="Non-Cap Fee",  
    description="Derived property for non-capitalised fees",  
    is_filterable=False
)

response = property_definition_api.create_derived_property_definition(
    create_derived_property_definition_request=property_definition
)
print(f"Property definition created with key: {response.key}")

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 15 Apr 2025 08:41:31 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-rate-limit-limit': '1m', 'x-rate-limit-remaining': '4997', 'x-rate-limit-reset': '2025-04-15T08:41:49.7226681Z', 'lusid-meta-success': 'False', 'lusid-meta-requestid': '2025041508-f2e3d3fd127a43fbbbceda92d5e381f7', 'lusid-meta-correlationid': '2025041508-f2e3d3fd127a43fbbbceda92d5e381f7', 'lusid-meta-duration': '66', 'x-envoy-upstream-service-time': '76', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagge

## 3. Create a Fees Side that uses the fee property

#### (Once functional, this can leverage the wildcard capability to access multiple fees in one side)

In [14]:
#creating a side for non-capitalised fees

side_definition = lm.SideDefinitionRequest(
    security = "Txn:LusidInstrumentId",
    currency = "Txn:TradeCurrency",
    rate = "Txn:TradeToPortfolioRate",
    units = "Transaction/Fees/NonCapFee1",
    amount = "Transaction/Fees/NonCapFee1",
)

response = transaction_config_api.set_side_definition(
    # Specify the name of the custom side
    side = custom_side,
    side_definition_request = side_definition
)
display(response)

{'amount': 'Transaction/Fees/NonCapFee1',
 'currency': 'Txn:TradeCurrency',
 'currentFace': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://nicolecam.lusid.com/app/insights/logs/2025041508-bd305663f22741bf82382f61ecdb6a60',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'notionalAmount': '0',
 'rate': 'Txn:TradeToPortfolioRate',
 'security': 'Txn:LusidInstrumentId',
 'side': 'TestSide',
 'units': 'Transaction/Fees/NonCapFee1'}

## 4. Create a "Buy"-style transaction type that includes a carry movement with the Fee Side

In [15]:
#Create new transaction type

transaction_type_definition = lm.TransactionTypeRequest(
    aliases = [
        lm.TransactionTypeAlias(          
            type = custom_transaction_type,
            description = "Purchase with cash balance reduced by Non-Cap Fees",
            transaction_class = "Fees",
            transaction_roles = "LongLonger"
        )
    ],
    movements = [
        # Replicate the first movement from the built-in Buy transaction type
        lm.TransactionTypeMovement(
            movement_types = "StockMovement",
            side = "Side1",
            direction = 1,
            name = "Increase instrument holding by the number of units in the standard way",
        ),
        # Replicate the second movement from the built-in Buy transaction type
        lm.TransactionTypeMovement(
            movement_types = "CashCommitment",
            side = "Side2",
            direction = -1,
            name = "Decrease cash balance by total cost in the standard way",
        ),
        # Create a third movement that uses the custom side
        lm.TransactionTypeMovement(            
            movement_types = "Carry",
            direction = -1,
            side = custom_side,
            name = "Additionally decrease cash balance by commission",
            properties = {},
        ),
    ],
    #Update total consideration calculation to include non ca[pitalised fees
    calculations = [
         lm.TransactionTypeCalculation(
              type = "DeriveTotalConsideration",
              formula = "Txn:GrossConsideration + Properties[Transaction/Fees/NonCapFee1]"
         )
    ]
)


response = transaction_config_api.set_transaction_type(
    source = f"{code}",
    type = custom_transaction_type,
    transaction_type_request = transaction_type_definition
)
display(transaction_config_api.get_transaction_type(source = f"{code}", type = custom_transaction_type))


{'aliases': [{'description': 'Purchase with cash balance reduced by Non-Cap '
                             'Fees',
              'isDefault': False,
              'transactionClass': 'Fees',
              'transactionRoles': 'LongLonger',
              'type': 'TestType'}],
 'calculations': [{'formula': 'Txn:GrossConsideration + '
                              'Properties[Transaction/Fees/NonCapFee1]',
                   'side': None,
                   'type': 'DeriveTotalConsideration'}],
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://nicolecam.lusid.com/app/insights/logs/2025041508-7e9506badecf451fa44a973c4d6c3d77',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'movements': [{'condition': '',
                'direction': 1,
                'mappings': [],
                'movementOptions': [],
                'movementTypes': 'StockMovement

## 5. Create Transactions

In [16]:
#Create transactions with this new type using the transactions data from previously 

transactions = []

for index, txn in transactions_df.iterrows():

    if txn["asset"] == "Cash":
        identifiers = {"Instrument/default/Currency": txn["currency"]}
        transaction_type = "FundsIn"
    else:
        identifiers = {"Instrument/default/Figi": txn["figi"]}
        transaction_type = custom_transaction_type

    transactions.append(
        lm.TransactionRequest(
            transaction_id = str(txn["txn_id"]),
            type = transaction_type,
            instrument_identifiers = identifiers,
            transaction_date = txn["trade_date"],
            settlement_date = txn["settle_date"],
            units = txn["units"],
            transaction_price = lm.TransactionPrice(price = txn["price"], type="Price"),
            total_consideration = lm.CurrencyAndAmount(
                amount = 0,
                currency = txn["currency"],
            ),
            properties = {},
            source = f"{code}"
        )
    )

upsert_transactions_response = transaction_portfolios_api.upsert_transactions(
    scope = scope, 
    code = code, 
    transaction_request = transactions
)

display(f"Transactions loaded at {str(upsert_transactions_response.version.as_at_date)}")
display(upsert_transactions_response)

'Transactions loaded at 2025-03-13 15:36:14.349811+00:00'

{'href': 'https://nicolecam.lusid.com/api/api/transactionportfolios/FeesTesting/FeesTestV1/transactions?asAt=2025-03-13T15%3A36%3A14.3498110%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://nicolecam.lusid.com/api/api/portfolios/FeesTesting/FeesTestV1?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2025-03-13T15%3A36%3A14.3498110%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://nicolecam.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://nicolecam.lusid.com/app/insights/logs/2025041508-dae988721ca0493bbcdb87db2925e732',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {},
 'ver

## Look at resultant holdings

In [17]:
# Get holdings for portfolio effective today
get_holdings_response=transaction_portfolios_api.get_holdings(
    scope = scope, 
    code = code,
    property_keys=["Instrument/default/Name"],
)

get_holdings_response_df=lusid_response_to_data_frame(get_holdings_response, rename_properties=True)
display(get_holdings_response_df)

,instrumentScope,instrumentUid,subHoldingKeys,properties.Instrument/default/Name.value.labelValue,properties.Instrument/default/Name.effectiveFrom,properties.Holding/default/SourcePortfolioId.value.labelValue,properties.Holding/default/SourcePortfolioId.effectiveFrom,properties.Holding/default/SourcePortfolioScope.value.labelValue,properties.Holding/default/SourcePortfolioScope.effectiveFrom,holdingType,units,settledUnits,cost.amount,cost.currency,costPortfolioCcy.amount,costPortfolioCcy.currency,currency,holdingTypeName,holdingId,notionalCost.amount,notionalCost.currency,amortisedCost.amount,amortisedCost.currency,amortisedCostPortfolioCcy.amount,amortisedCostPortfolioCcy.currency,variationMargin.amount,variationMargin.currency,variationMarginPortfolioCcy.amount,variationMarginPortfolioCcy.currency,settlementSchedule
0,FeesTestingFeesTestV1,LUID_00003D68,{},BP,0001-01-01 00:00:00+00:00,FeesTestV1,0001-01-01 00:00:00+00:00,FeesTesting,0001-01-01 00:00:00+00:00,P,10.00,10.00,5.00,GBP,5.00,GBP,GBP,Position,75525627,0.00,GBP,5.00,GBP,5.00,GBP,0.00,GBP,0.00,GBP,[]
1,default,CCY_GBP,{},GBP,0001-01-01 00:00:00+00:00,FeesTestV1,0001-01-01 00:00:00+00:00,FeesTesting,0001-01-01 00:00:00+00:00,B,495.00,495.00,495.00,GBP,495.00,GBP,GBP,Balance,75525626,0.00,GBP,495.00,GBP,495.00,GBP,0.00,GBP,0.00,GBP,[]
